In [1]:
import cv2
import pytesseract
from datetime import datetime, timedelta
import re
import os
import matplotlib.pyplot as plt

In [2]:
# Load the video and timestamps
video_path = "./build/macosx/arm64/debug/long-record-04.mp4"
timestamps_path = os.path.splitext(video_path)[0] + ".txt"

cap = cv2.VideoCapture(video_path)
with open(timestamps_path, "r") as f:
    timestamps = [float(line.strip()) for line in f.readlines()]

In [3]:
frame_index = -1
timestamp_prev = None
frame_indices = []
diffs = []

In [4]:
while True:
    # Read a frame from the video
    ret, frame = cap.read()
    frame_index += 1
    if not ret:
        break

    # Convert the frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Apply thresholding to preprocess the frame for better OCR results
    thresh = cv2.threshold(gray, 150, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]

    # Define the region of interest (ROI) where the timestamp is located (right upper part)
    # Adjust these coordinates according to your video
    frame_height, frame_width = frame.shape[:2]
    x, y, w, h = frame_width - 370, 70, 270, 70
    roi = thresh[y : y + h, x : x + w]

    # Use pytesseract to extract text from the ROI
    # custom_config = r"--oem 3 --psm 6"  # OEM 3 is the default OCR Engine Mode, PSM 6 is for a single block of text
    custom_config = r"--psm 6 -c tessedit_char_whitelist=0123456789:"
    timestamp_text = pytesseract.image_to_string(roi, config=custom_config)
    timestamp_text = re.sub(r"[^0-9:\-\s]", "", timestamp_text).strip()

    # Parse the timestamp text to a datetime object
    try:
        timestamp = datetime.strptime(timestamp_text, "%H:%M:%S")
        if timestamp != timestamp_prev:
            # get the current readed timestamp
            unixtime = timestamps[frame_index]
            # parse the timestamp to a datetime object
            timestamp_read = datetime.fromtimestamp(unixtime)
            # convert timezone, subtract 2 hours from the readed timestamp
            timestamp_read = timestamp_read - timedelta(hours=2)
            timestamp_prev = timestamp

            # the timestamp ocred has no data, so we get the date from timestamp
            date = timestamp_read.date()
            # add the date to the timestamp
            timestamp = timestamp.replace(
                year=date.year, month=date.month, day=date.day
            )
            print(
                f"Frame {frame_index}: OCR Timestamp: {timestamp}, Readed Timestamp: {timestamp_read}"
            )

            # calculate the difference in milliseconds
            diff = timestamp_read.timestamp() - timestamp.timestamp()
            # convert seconds to milliseconds, keep only the integer part
            diff = int(diff * 1000)
            print(f"Frame {frame_index}: Difference: {diff}")
            frame_indices.append(frame_index)
            diffs.append(diff)

    except ValueError:
        print(f"Frame {frame_index}: Failed to parse timestamp: {timestamp_text}")

    # Display the frame and ROI for debugging purposes
    # cv2.imshow("Frame", frame)
    # cv2.imshow("Thresholded ROI", roi)

    # Exit on pressing 'q'
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

Frame 0: OCR Timestamp: 2024-10-22 10:34:39, Readed Timestamp: 2024-10-22 10:34:39.393999
Frame 0: Difference: 393
Frame 20: OCR Timestamp: 2024-10-22 10:34:40, Readed Timestamp: 2024-10-22 10:34:40.060999
Frame 20: Difference: 60
Frame 49: OCR Timestamp: 2024-10-22 10:34:49, Readed Timestamp: 2024-10-22 10:34:41.026999
Frame 49: Difference: -7973
Frame 108: OCR Timestamp: 2024-10-22 10:34:43, Readed Timestamp: 2024-10-22 10:34:43.079999
Frame 108: Difference: 79
Frame 137: OCR Timestamp: 2024-10-22 10:34:44, Readed Timestamp: 2024-10-22 10:34:44.080999
Frame 137: Difference: 80
Frame 166: OCR Timestamp: 2024-10-22 10:34:45, Readed Timestamp: 2024-10-22 10:34:45.080999
Frame 166: Difference: 80
Frame 195: OCR Timestamp: 2024-10-22 10:34:46, Readed Timestamp: 2024-10-22 10:34:46.081999
Frame 195: Difference: 81
Frame 224: OCR Timestamp: 2024-10-22 10:34:47, Readed Timestamp: 2024-10-22 10:34:47.084999
Frame 224: Difference: 84
Frame 253: OCR Timestamp: 2024-10-22 10:34:48, Readed Timest

In [5]:
# Release the video capture object and close all windows
cap.release()
cv2.destroyAllWindows()

In [1]:
# Plot the difference between OCR and read timestamps
plt.figure(figsize=(10, 5))
plt.plot(frame_indices, diffs, marker='o', linestyle='-', color='b')
plt.xlabel('Frame Index')
plt.ylabel('Time Difference (ms)')
# plt.ylim(0, 400)
plt.title('Time Difference Between OCR Timestamp and Read Timestamp')
plt.grid(True)
plt.savefig('timestamp_difference_plot.png')
plt.show()

NameError: name 'plt' is not defined